In [5]:
%load_ext autoreload
%autoreload 2

import pyspiel
import open_spiel.python.examples.ubc_utils
from open_spiel.python.visualizations import ubc_treeviz
from open_spiel.python.examples.ubc_utils import *
from auctions.webutils import *
from absl import logging
logging.set_verbosity(logging.INFO)
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from open_spiel.python.examples.ubc_cma import *
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator, TremblingAgentDecorator, ModalAgentDecorator

In [26]:
run = EquilibriumSolverRun.objects.get(name='jun9_jun9_0_base-cfr_outcomemccfr_outcome-101')
game, checkpoint, policy = get_results(run)

c = checkpoint.equilibrium_solver_run.config
cfr = c.get('solver_type') == 'cfr'

env_and_policy = ppo_db_checkpoint_loader(checkpoint, cfr=cfr)
for player in range(game.num_players()):
    env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
policy = env_and_policy.make_policy()

# Best Responses + NashConv

In [27]:
policy

In [32]:
state = game.new_initial_state().child(0).child(0)

print(policy.action_probabilities(state))
print(policy.action_probabilities(state.child(9)))
print(policy.action_probabilities(state.child(9).child(9)))
print(policy.action_probabilities(state.child(9).child(9).child(6)))
print(policy.action_probabilities(state.child(9).child(9).child(6).child(8)))

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 1.0}
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 1.0}
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 1.0, 7: 0.0, 8: 0.0, 9: 0.0}
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 1.0, 9: 0.0}
{0: 0.0, 1: 0.0, 2: 0.0, 5: 0.0, 6: 1.0}


In [29]:
from open_spiel.python.algorithms.best_response import BestResponsePolicy
for player_num in range(game.num_players()):
    print("Initializing best-responder...")
    br = BestResponsePolicy(game, player_num, policy)
    print("Computing best reponse...")
    print(br.value(game.new_initial_state()))

Initializing best-responder...
Computing best reponse...
110.2971
Initializing best-responder...
Computing best reponse...
214.127


In [27]:
from open_spiel.python.algorithms.exploitability import nash_conv
nash_conv(game, policy, True)

48.963800000000006

# External Sampling CFR

In [22]:
run = EquilibriumSolverRun.objects.get(name='jun9_jun9_0_base-cfr_outcomemccfr_outcome-101')
game, checkpoint, policy = get_results(run)

In [23]:
from open_spiel.python.algorithms.external_sampling_mccfr import ExternalSamplingSolver

In [24]:
solver = ExternalSamplingSolver(game)

In [25]:
for i in tqdm(range(1000)):
    solver.iteration()

  0%|                                                                                                                                                                            | 0/1000 [00:00<?, ?it/s]

 31%|█████████████████████████████████████████████████▌                                                                                                                | 306/1000 [03:28<07:53,  1.47it/s]


KeyboardInterrupt: 

In [ ]:
1 * 3600 * 12 / 1e6

1.08